In [3]:
import pandas as pd

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torchmetrics.regression import R2Score

In [7]:
DATA_PATH = '../Data/수도전기사용량_특정호수연속값.xlsx'

df1 = pd.read_excel(DATA_PATH, header = 0)
df2 = pd.read_excel(DATA_PATH, header = 0, sheet_name = '전기사용량')

df1

,세대,2023-06-01 00:00-06:00,2023-06-01 06:00-12:00,2023-06-01 12:00-18:00,2023-06-01 18:00-24:00,2023-06-02 00:00-06:00,2023-06-02 06:00-12:00,2023-06-02 12:00-18:00,2023-06-02 18:00-24:00,2023-06-03 00:00-06:00,...,2023-06-28 12:00-18:00,2023-06-28 18:00-24:00,2023-06-29 00:00-06:00,2023-06-29 06:00-12:00,2023-06-29 12:00-18:00,2023-06-29 18:00-24:00,2023-06-30 00:00-06:00,2023-06-30 06:00-12:00,2023-06-30 12:00-18:00,2023-06-30 18:00-24:00
0,1호,342,257,163,333,240,294,29,224,107,...,244,7,109,226,44,256,407,311,163,272
1,2호,228,151,398,208,353,204,336,292,57,...,341,40,291,6,323,193,123,417,282,153
2,3호,175,194,56,398,304,166,107,410,424,...,316,4,129,201,306,417,298,104,116,318
3,4호,50,11,104,309,46,37,337,394,260,...,315,156,401,179,68,80,347,444,333,391
4,5호,377,359,75,165,421,279,313,432,77,...,137,135,182,33,312,228,200,289,324,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96호,437,416,267,3,64,183,11,340,39,...,17,288,331,200,200,361,203,7,28,262
96,97호,254,130,249,162,331,17,322,431,370,...,408,408,59,166,310,241,275,71,204,351
97,98호,382,47,48,201,248,197,98,224,299,...,349,18,235,262,26,296,317,417,135,350
98,99호,30,166,1,182,376,49,333,28,435,...,254,353,376,290,246,310,224,437,222,252


In [8]:
df2

,세대,2023-06-01 00:00-06:00,2023-06-01 06:00-12:00,2023-06-01 12:00-18:00,2023-06-01 18:00-24:00,2023-06-02 00:00-06:00,2023-06-02 06:00-12:00,2023-06-02 12:00-18:00,2023-06-02 18:00-24:00,2023-06-03 00:00-06:00,...,2023-06-28 12:00-18:00,2023-06-28 18:00-24:00,2023-06-29 00:00-06:00,2023-06-29 06:00-12:00,2023-06-29 12:00-18:00,2023-06-29 18:00-24:00,2023-06-30 00:00-06:00,2023-06-30 06:00-12:00,2023-06-30 12:00-18:00,2023-06-30 18:00-24:00
0,1호,1.183566,1.098491,1.926311,1.818524,1.389089,1.101629,1.555929,1.437666,1.131855,...,1.258612,1.571295,1.431408,1.568614,1.660112,1.798504,1.503162,1.718257,1.742305,1.535451
1,2호,1.234794,1.411516,1.154392,1.170356,1.430910,1.227085,1.001902,1.839712,1.460300,...,1.033758,1.140594,1.482664,1.849836,1.725604,1.154982,1.615537,1.093102,1.929949,1.031971
2,3호,1.853471,1.219136,1.844583,1.277626,1.901040,1.235121,1.890324,1.070304,1.335981,...,1.746917,1.817748,1.511441,1.078073,1.481758,1.470617,1.920808,1.731622,1.904152,1.355065
3,4호,1.936965,1.535739,1.371238,1.109464,1.508162,1.087476,1.871968,1.267080,1.021641,...,1.309037,1.278825,1.629540,1.952493,1.311932,1.040236,1.741358,1.083411,1.801820,1.331651
4,5호,1.423212,1.140493,1.114285,1.095283,1.324076,1.099512,1.984252,1.163054,1.502168,...,1.153057,1.543684,1.503237,1.916176,1.957507,1.791857,1.589837,1.887428,1.829302,1.937173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96호,1.459120,1.991922,1.548038,1.793713,1.878990,1.733811,1.547036,1.495122,1.602313,...,1.662377,1.643675,1.470646,1.513499,1.325816,1.636515,1.722546,1.543375,1.205620,1.960089
96,97호,1.982808,1.500700,1.891048,1.362217,1.663813,1.012351,1.356026,1.335531,1.585381,...,1.745376,1.192339,1.242801,1.992296,1.485609,1.607751,1.821993,1.822925,1.960104,1.577438
97,98호,1.619027,1.834624,1.473008,1.587140,1.939395,1.045415,1.779117,1.828781,1.661731,...,1.660698,1.135158,1.849446,1.512724,1.778517,1.893890,1.125848,1.834702,1.228773,1.129060
98,99호,1.050792,1.552936,1.410190,1.354796,1.139832,1.024649,1.144882,1.799458,1.387399,...,1.985108,1.231563,1.493250,1.447089,1.588705,1.286219,1.396888,1.522067,1.490977,1.116907


In [4]:
df1.drop(1001, axis = 0, inplace = True)


In [5]:
df1.dropna(axis = 1, inplace = True)
df2.dropna(axis = 1, inplace = True)

df1

,Unnamed: 0,2023-06-01 00:00:00,2023-06-01 00:00:00.1,2023-06-01 00:00:00.2,2023-06-01 00:00:00.3,2023-06-02 00:00:00,2023-06-02 00:00:00.1,2023-06-02 00:00:00.2,2023-06-02 00:00:00.3,2023-06-03 00:00:00,...,2023-08-29 00:00:00.2,2023-08-29 00:00:00.3,2023-08-30 00:00:00,2023-08-30 00:00:00.1,2023-08-30 00:00:00.2,2023-08-30 00:00:00.3,2023-08-31 00:00:00,2023-08-31 00:00:00.1,2023-08-31 00:00:00.2,2023-08-31 00:00:00.3
0,세대,1.00,2.00,3.00,4.00,1.00,2.00,3.00,4.00,1.00,...,3.00,4.00,1.00,2.00,3.00,4.00,1.00,2.00,3.00,4.00
1,1호,1.23,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.30,...,1.66,1.64,1.27,1.29,1.69,1.57,1.26,1.36,1.64,1.59
2,2호,1.35,1.35,1.35,1.34,1.36,1.42,1.41,1.38,1.36,...,1.59,1.48,1.41,1.47,1.54,1.48,1.41,1.47,1.64,1.51
3,3호,1.36,1.44,1.42,1.36,1.36,1.42,1.43,1.39,1.32,...,1.64,1.79,1.34,1.35,1.46,1.49,1.35,1.42,1.47,1.36
4,4호,1.23,1.44,1.73,1.61,1.33,1.47,1.73,1.71,1.32,...,1.73,1.71,1.32,1.49,1.94,1.68,1.23,1.39,1.65,1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,996호,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.30,1.49,...,1.68,1.36,1.43,1.66,1.79,1.44,1.64,1.54,1.46,1.34
997,997호,1.29,1.69,1.57,1.26,1.36,1.64,1.59,1.48,1.72,...,1.46,1.36,1.45,1.69,1.79,1.35,1.43,1.69,1.75,1.44
998,998호,1.44,1.64,1.55,1.45,1.56,1.45,1.71,1.63,1.34,...,1.79,1.35,1.35,1.46,1.49,1.25,1.45,1.73,1.62,1.35
999,999호,1.23,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.32,...,1.49,1.25,1.45,1.73,1.62,1.23,1.24,1.26,1.57,1.34


In [6]:
df2.info

<bound method DataFrame.info of      Unnamed: 0  2023-06-01 00:00:00  2023-06-01 00:00:00.1  \
0            세대                    1                      2   
1            1호                   30                    120   
2            2호                   58                    218   
3            3호                   28                    184   
4            4호                   14                    169   
...         ...                  ...                    ...   
996        996호                  188                     95   
997        997호                  188                     95   
998        998호                  218                    350   
999        999호                   43                    136   
1000      1000호                   10                     68   

      2023-06-01 00:00:00.2  2023-06-01 00:00:00.3  2023-06-02 00:00:00  \
0                         3                      4                    1   
1                       210                    410           

In [9]:
df1.drop(df1.columns[0], axis = 1, inplace = True)
df2.drop(df2.columns[0], axis = 1, inplace = True)

In [12]:
column_name = pd.date_range('2023-06-01', '2023-07-1', freq = '6H')
column_name = column_name[:-1]

column_name

DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 06:00:00',
               '2023-06-01 12:00:00', '2023-06-01 18:00:00',
               '2023-06-02 00:00:00', '2023-06-02 06:00:00',
               '2023-06-02 12:00:00', '2023-06-02 18:00:00',
               '2023-06-03 00:00:00', '2023-06-03 06:00:00',
               ...
               '2023-06-28 12:00:00', '2023-06-28 18:00:00',
               '2023-06-29 00:00:00', '2023-06-29 06:00:00',
               '2023-06-29 12:00:00', '2023-06-29 18:00:00',
               '2023-06-30 00:00:00', '2023-06-30 06:00:00',
               '2023-06-30 12:00:00', '2023-06-30 18:00:00'],
              dtype='datetime64[ns]', length=120, freq='6H')

In [13]:
df1.columns = column_name
df2.columns = column_name

df1.columns

DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 06:00:00',
               '2023-06-01 12:00:00', '2023-06-01 18:00:00',
               '2023-06-02 00:00:00', '2023-06-02 06:00:00',
               '2023-06-02 12:00:00', '2023-06-02 18:00:00',
               '2023-06-03 00:00:00', '2023-06-03 06:00:00',
               ...
               '2023-06-28 12:00:00', '2023-06-28 18:00:00',
               '2023-06-29 00:00:00', '2023-06-29 06:00:00',
               '2023-06-29 12:00:00', '2023-06-29 18:00:00',
               '2023-06-30 00:00:00', '2023-06-30 06:00:00',
               '2023-06-30 12:00:00', '2023-06-30 18:00:00'],
              dtype='datetime64[ns]', length=120, freq='6H')

In [10]:
df1[df1.columns[0:12]]

,2023-06-01 00:00:00,2023-06-01 06:00:00,2023-06-01 12:00:00,2023-06-01 18:00:00,2023-06-02 00:00:00,2023-06-02 06:00:00,2023-06-02 12:00:00,2023-06-02 18:00:00,2023-06-03 00:00:00,2023-06-03 06:00:00,2023-06-03 12:00:00,2023-06-03 18:00:00
0,1.00,2.00,3.00,4.00,1.00,2.00,3.00,4.00,1.00,2.00,3.00,4.00
1,1.23,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.30,1.49,1.94,1.68
2,1.35,1.35,1.35,1.34,1.36,1.42,1.41,1.38,1.36,1.42,1.48,1.39
3,1.36,1.44,1.42,1.36,1.36,1.42,1.43,1.39,1.32,1.41,1.49,1.41
4,1.23,1.44,1.73,1.61,1.33,1.47,1.73,1.71,1.32,1.49,1.94,1.68
...,...,...,...,...,...,...,...,...,...,...,...,...
996,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.30,1.49,1.94,1.68,1.26
997,1.29,1.69,1.57,1.26,1.36,1.64,1.59,1.48,1.72,1.63,1.37,1.43
998,1.44,1.64,1.55,1.45,1.56,1.45,1.71,1.63,1.34,1.31,1.48,1.54
999,1.23,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.32,1.49,1.94,1.68


In [11]:
df1.drop(0, axis = 0, inplace = True)
df2.drop(0, axis = 0, inplace = True)

df1

,2023-06-01 00:00:00,2023-06-01 06:00:00,2023-06-01 12:00:00,2023-06-01 18:00:00,2023-06-02 00:00:00,2023-06-02 06:00:00,2023-06-02 12:00:00,2023-06-02 18:00:00,2023-06-03 00:00:00,2023-06-03 06:00:00,...,2023-08-29 12:00:00,2023-08-29 18:00:00,2023-08-30 00:00:00,2023-08-30 06:00:00,2023-08-30 12:00:00,2023-08-30 18:00:00,2023-08-31 00:00:00,2023-08-31 06:00:00,2023-08-31 12:00:00,2023-08-31 18:00:00
1,1.23,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.30,1.49,...,1.66,1.64,1.27,1.29,1.69,1.57,1.26,1.36,1.64,1.59
2,1.35,1.35,1.35,1.34,1.36,1.42,1.41,1.38,1.36,1.42,...,1.59,1.48,1.41,1.47,1.54,1.48,1.41,1.47,1.64,1.51
3,1.36,1.44,1.42,1.36,1.36,1.42,1.43,1.39,1.32,1.41,...,1.64,1.79,1.34,1.35,1.46,1.49,1.35,1.42,1.47,1.36
4,1.23,1.44,1.73,1.61,1.33,1.47,1.73,1.71,1.32,1.49,...,1.73,1.71,1.32,1.49,1.94,1.68,1.23,1.39,1.65,1.61
5,1.22,1.35,1.67,1.57,1.31,1.48,1.73,1.76,1.33,1.44,...,1.66,1.64,1.27,1.29,1.69,1.57,1.26,1.36,1.64,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.30,1.49,1.94,...,1.68,1.36,1.43,1.66,1.79,1.44,1.64,1.54,1.46,1.34
997,1.29,1.69,1.57,1.26,1.36,1.64,1.59,1.48,1.72,1.63,...,1.46,1.36,1.45,1.69,1.79,1.35,1.43,1.69,1.75,1.44
998,1.44,1.64,1.55,1.45,1.56,1.45,1.71,1.63,1.34,1.31,...,1.79,1.35,1.35,1.46,1.49,1.25,1.45,1.73,1.62,1.35
999,1.23,1.45,1.73,1.62,1.31,1.48,1.74,1.71,1.32,1.49,...,1.49,1.25,1.45,1.73,1.62,1.23,1.24,1.26,1.57,1.34


In [14]:
def slicing(data, step, max_length, num):

    df = pd.DataFrame(columns = range(0, step + 1))

    diff = max_length - step
    
    for i in range(num):
        for j in range(diff):
            df.loc[i * diff + j] = list(data.iloc[i][data.columns[j : j + step + 1]])
    
    return df


In [15]:
electric_df = slicing(df1, 28, 30, 100)
electric_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,342,257,163,333,240,294,29,224,107,89,...,208,265,368,39,127,231,399,232,102,49
1,257,163,333,240,294,29,224,107,89,328,...,265,368,39,127,231,399,232,102,49,143
2,228,151,398,208,353,204,336,292,57,319,...,373,57,272,139,276,349,325,91,91,386
3,151,398,208,353,204,336,292,57,319,129,...,57,272,139,276,349,325,91,91,386,356
4,175,194,56,398,304,166,107,410,424,315,...,72,362,71,181,276,294,404,444,283,378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,47,48,201,248,197,98,224,299,89,131,...,212,183,450,355,344,300,253,101,211,127
196,30,166,1,182,376,49,333,28,435,347,...,292,172,41,382,42,117,23,59,399,114
197,166,1,182,376,49,333,28,435,347,213,...,172,41,382,42,117,23,59,399,114,393
198,303,427,286,144,63,9,196,410,375,86,...,23,397,118,384,303,143,8,13,245,378


In [95]:
electric_df.to_csv('./Data/electric_df.csv')

In [96]:
water_df = slicing(df2, 28, 368, 1000)
water_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,30,120,210,410,32,184,180,260,35,145,...,95,46,139,204,198,53,162,210,150,51
1,120,210,410,32,184,180,260,35,145,203,...,46,139,204,198,53,162,210,150,51,169
2,210,410,32,184,180,260,35,145,203,216,...,139,204,198,53,162,210,150,51,169,204
3,410,32,184,180,260,35,145,203,216,43,...,204,198,53,162,210,150,51,169,204,169
4,32,184,180,260,35,145,203,216,43,136,...,198,53,162,210,150,51,169,204,169,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339995,157,230,169,47,122,218,350,110,183,190,...,58,218,290,105,10,68,38,263,78,158
339996,230,169,47,122,218,350,110,183,190,230,...,218,290,105,10,68,38,263,78,158,290
339997,169,47,122,218,350,110,183,190,230,98,...,290,105,10,68,38,263,78,158,290,300
339998,47,122,218,350,110,183,190,230,98,143,...,105,10,68,38,263,78,158,290,300,280


In [97]:
water_df.to_csv('./Data/water_df.csv')

In [ ]:
electric_df2 = slicing(df1, 56, 368, 1000)
water_df2 = slicing(df2, 56, 368, 1000)

electric_df2.to_csv('./Data/electric_df_2_weeks.csv')
water_df2.to_csv('./Data/water_df_2_weeks.csv')

: 